In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
data = [(1, "Commentary 1"), (2, "Commentary 2")]
columns = ["id", "text"]

df = spark.createDataFrame(data, columns)


In [0]:
delta_path = "/mnt/external_data/delta_commentary"
dbutils.fs.rm(delta_path, recurse=True)
df.write.format("delta").save(delta_path)
spark.sql(f"DROP TABLE IF EXISTS commentary_delta")
spark.sql(f"CREATE TABLE commentary_delta USING DELTA LOCATION '{delta_path}'")


Out[2]: DataFrame[]

In [0]:
df_updated.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(delta_path)


In [0]:
%sql
DESCRIBE HISTORY commentary_delta;


version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2025-04-15T10:06:28.000+0000,8517804981882898,santhiyasri394@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1462301345730024),0415-092308-72nftrx8,0,WriteSerializable,false,"Map(numFiles -> 2, numOutputRows -> 2, numOutputBytes -> 2224)",null,Databricks-Runtime/12.2.x-scala2.12
0,2025-04-15T10:05:44.000+0000,8517804981882898,santhiyasri394@gmail.com,WRITE,"Map(mode -> ErrorIfExists, partitionBy -> [])",null,List(1462301345730024),0415-092308-72nftrx8,null,WriteSerializable,true,"Map(numFiles -> 2, numOutputRows -> 2, numOutputBytes -> 1780)",null,Databricks-Runtime/12.2.x-scala2.12


In [0]:
%sql
SELECT * FROM commentary_delta;


id,text,dummy
1,Commentary 1,v2
2,Commentary 2,v2


In [0]:
%sql
SELECT * FROM delta.`/mnt/external_data/delta_commentary` VERSION AS OF 0;


id,text
1,Commentary 1
2,Commentary 2


In [0]:
spark.read.table("commentary_delta").printSchema()


root
 |-- id: long (nullable = true)
 |-- text: string (nullable = true)
 |-- dummy: string (nullable = true)

